In [1]:
# Dependencies
import pandas as pd
import requests
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter 
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/jpot/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


# NASA Mars News

In [3]:
# define news url
mars_news_url = "https://mars.nasa.gov/news/"
browser.visit(mars_news_url)

# create beautiful soup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# retrieve news article title and paragraph
article = soup.find("div", class_='list_text')
news_title = article.find("div", class_="content_title").text
news_p = article.find("div", class_ ="article_teaser_body").text
print(f"The latest article title is: \n{news_title}")
print()
print(f"The descriptive paragraph is: \n{news_p}")

The latest article title is: 
Nearly 11 Million Names of Earthlings are on Mars Perseverance

The descriptive paragraph is: 
When the Perseverance rover safely touched down on the Martian surface, inside Jezero Crater, on Feb. 18, 2021, it was also a safe landing for the nearly 11 million names on board.


# JPL Mars Space Images

In [4]:
# define mars images url
mars_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(mars_image_url)

# create beautiful soup object
html = browser.html
images_soup = BeautifulSoup(html, 'html.parser')

In [8]:
# retrieve featured image link
main_url = "https://www.jpl.nasa.gov"
relative_image_path = images_soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]
featured_image_url = main_url + relative_image_path
featured_image_url

TypeError: 'NoneType' object is not subscriptable

# Mars Facts

In [9]:
# define mars facts url
mars_facts_url = "https://space-facts.com/mars/"
browser.visit(mars_facts_url)

In [10]:
# Use Pandas to scrape the table containing facts about Mars
tables = pd.read_html(mars_facts_url)
mars_facts = tables[0]

# Rename the columns
mars_facts.columns= ['Description', 'Value']

# Reset the index
mars_facts.set_index('Description', inplace=True)
mars_facts

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [11]:
# Use Pandas to convert the table into a HTML table string
mars_facts.to_html('table.html')

# Mars Hemispheres

In [12]:
# define mars hemispheres url
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)

# create beautiful soup object
html = browser.html
hemisphere_soup = BeautifulSoup(html, 'html.parser')

In [13]:

# Create a list of dictionaries to store titles & links to images
hemisphere_image_urls = []

# Retrieve all elements that contain image information
results = hemisphere_soup.find("div", class_ = "result-list" )
hemispheres = results.find_all("div", class_="item")

# Iterate through each image
for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})

# Print image title and url
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]